<a href="https://colab.research.google.com/github/gonzalobarrazueta/test/blob/main/MAPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Crear diccionario de calles**
El archivo .csv contiene el `id` que le corresponde a cada calle. Guardo los datos en un diccionario llamado `streetsDicitonary`

In [16]:
import csv
import json

In [17]:
def addKey_Values(dictionary, key, value):
    dictionary[key] = value

In [18]:
def cargarDatos(dictionary):
    with open('Calles de San Martín de Porres.csv') as file: 
        reader = csv.reader(file)
        
        streetId = 0
        for row in reader:
            addKey_Values(dictionary, row[0].lower(), streetId)
            streetId += 1

In [19]:
def createStreetsDictionary():
    streets = {}
    # carga datos al diccionario 'streets'
    cargarDatos(streets)

    streets_pretty = json.dumps(streets, indent=2)
    print(streets_pretty)
    
    return streets

In [ ]:
streetsDictionary = createStreetsDictionary()

###**Crear Lista de Adyacencia para las calles y sus intersecciones**
* -1 significa que la calle aún no está mapeada



In [21]:
def getLastValueInDictionary():
    lastStreet = list(streetsDictionary.keys())[-1]
    id = streetsDictionary[lastStreet]
    return id

In [22]:
def streetExists(streetName):
    if streetName in streetsDictionary:
        s = streetsDictionary[streetName]
    else:
        s = -1
        # agrega la calle al final del diccionario, si no existe
        #s = getLastValueInDictionary() + 1
        #streetsDictionary[streetName] = s
    return s

In [23]:
def getAmountOfIntersections(graph):
    totalIntersections = 0
    for intersections in graph:
        for i in range(len(intersections)):
            if (intersections[i] != -1):
                totalIntersections += 1
        #totalIntersections += len(intersections)
    
    print(totalIntersections)

def printAdList(graph):
    print(f"n = {len(intersectionsAdList)}")
    i = 0
    for intersection in intersectionsAdList:
        print(i, intersection)
        i += 1

In [ ]:
# adjacencyList: arreglo de arreglos, donde la pos del primer elemento (1er arreglo)
# es la calle y el arreglo son todas las calles con las que se intersecta
def createAdjacencyList(adjacencyList):
    with open('Calles de San Martín de Porres.csv') as file: 
        reader = csv.reader(file)

        # Leo línea por línea
        for row in reader:  
            intersections = []
            for i in range(len(row)):
                if i != 0:
                    intersections.append(streetExists(row[i].lower()))

            adjacencyList.append(intersections)

    #for intersection in adjacencyList:
    #    print(intersection, end='\n')
    return adjacencyList

intersectionsAdList = createAdjacencyList([])

getAmountOfIntersections(intersectionsAdList)
printAdList(intersectionsAdList)

###**Lista de adyacencia de las intersecciones de intersecciones**

In [26]:
# sirve para crear las intersecciones, como keys, en un diccionario
# si las keys aún no han sido creadas, las agrega al diccionario
def createKeys(graph):

    intersectionsIDs = {}

    for i in range(len(graph)):
        for street in graph[i]:
            if (i, street) not in intersectionsIDs and street != -1:
                intersectionsIDs[(i, street)] = []

    return intersectionsIDs

In [27]:
def addIntersections(graph, i, j):  
    
    street = graph[i][j]

    if street != -1:
        # agrego 1 a la izquierda
        if j > 0 and graph[i][j - 1] != -1:
            tup = (i, graph[i][j - 1])
            map[(i, street)].append(tup)

        # agrego 1 a la derecha
        if j < len(graph[i]) - 1 and graph[i][j + 1] != -1:
            tup = (i, graph[i][j + 1])
            map[(i, street)].append(tup)

In [28]:
def addIntersectionsReversed(graph, i, j):  
    #    0  1   2 
    # 0 [3, 5, 17]

    # 3 [7, 14, 0, 25]

    street = graph[i][j] # street = 3

    # ahora voy al reverso de la lista

    if i != -1 and street != -1:
        #preguntar si i existe en 3
        if (street < len(graph)) and (i in graph[street]):
            tup = (street, i)
            map[(i, street)].append(tup)
            
            # encuentro el índice de 0
            k = graph[street].index(i) # k = 2

            if k > 0 and graph[street][k - 1] != -1:
                tup = (street, graph[street][k - 1])
                map[(i, street)].append(tup)
        
            if k < len(graph[street]) - 1 and graph[street][k + 1] != -1:
                tup = (street, graph[street][k + 1])
                map[(i, street)].append(tup)

In [29]:
def joinIntersections(graph):
    n = len(graph)
    
    for i in range(n):
        
        for j in range(len(graph[i])):

            addIntersections(graph, i, j)
            addIntersectionsReversed(graph, i, j)

In [ ]:
# Diccionario que va a guardar la unión de todas las intersecciones
map = dict()
# crea las llaves en el diccionario "map"
map = createKeys(intersectionsAdList)

joinIntersections(intersectionsAdList)

amountNodes = 0 

for key, value in map.items():
    amountNodes += len(value)
    print(key, ":", value)

#print(f"Cantidad de nodos: {amountNodes}")

###**Write to file**

In [ ]:
def latAndlong():
    with open("latlong.txt", "w") as f:
        with open('Calles de San Martín de Porres.csv') as file: 
            reader = csv.reader(file)

            for row in reader:
                for i in range(1, len(row)):                    
                    f.write(f"{row[0]}, {row[i]}\n")    

latAndlong()

In [ ]:
# Write Streets Dictionary
def writeStreetsDictionary():
    with open("streetNames.txt", "w") as f:
        for key, value in streetsDictionary.items():
            f.write(key + "\n")

writeStreetsDictionary()

In [ ]:
# Write Intersections Adjacency List
def writeStreetsAdList():
    with open("streetsAdList.txt", "w") as f:
        for streets in intersectionsAdList:
            for i in range(len(streets)):
                streets[i] = str(streets[i])
            
            f.write(', '.join(streets) + "\n")
        
writeStreetsAdList()

In [ ]:
# Write street map
def writeStreetMap():
    with open("map.txt", "w") as f:
        
        for key, value in map.items():
            listWOCommas = (str(value).strip('[]')).replace(",", "")
            f.write(str(key) + " " + listWOCommas + '\n')
            
writeStreetMap()